In [1]:
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam
from keras import layers, callbacks

from keras.utils import to_categorical

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

from tensorflow.python.keras.utils.vis_utils import plot_model
import pydot

from scipy.stats import norm
from scipy import stats
import os

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
import dataframe_image as dfi
import pickle
import math

from keras.utils.layer_utils import count_params


2023-05-21 09:39:24.589610: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 09:39:24.615924: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sfs = [1, 2, 4, 6, 8, 16, 32]
layer_options = [2, 3]
weight_options = [0, 1, 2] #options
extra_options = ['a', 'b']
nr_classes = 3
channel_width = 79

### Gotta load in some data

In [3]:
datapath = 'All_data/'
labelpath = 'All_labels/'
data_list = os.listdir(datapath)
#print(data_list)

#all_data = [] #if we want to have data and labels in one list
complete_data = []
complete_labels = []

total_channels = 79
total_scans_pr_sample = 1


for csv_file in data_list:
    data_file = datapath + csv_file
    current_data_file = pd.read_csv(data_file,header=None)

    label_file = labelpath + csv_file
    label_file = label_file.replace('.csv', '_labels.csv')
    current_label_file = pd.read_csv(label_file,header=None)

    for data_iter in range(len(current_data_file.index)):
        #Pulling out the data from a row and putting it in the list
        current_data_point = np.array(current_data_file.iloc[data_iter])
        #current_data_point = current_data_point.reshape(total_scans_pr_sample,total_channels)
        complete_data.append(current_data_point)
        
        #adding the label to the datamatrix as the last row
        label_row = np.array(current_label_file.iloc[data_iter])
        #label_row = label_row.reshape(1,total_channels)
        complete_labels.append(label_row)
        
        #all_data.append(np.vstack([current_data_point, label_row])) #if we want to have data and labels in one list

        
data_train, data_test, labels_train, labels_test = train_test_split(complete_data, complete_labels, train_size=0.8, random_state=112)

data_train = np.array(data_train)
data_test = np.array(data_test)
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)
print(labels_test.shape)

scaler = preprocessing.StandardScaler().fit(data_train)

# scale everything using that scaler
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

labels_test = to_categorical(labels_test)
labels_train = to_categorical(labels_train)
print(labels_test.shape)

(58140, 79)
(58140, 79, 3)


In [4]:
def weighted_mean_squared_error(class_weight):
  def loss(y_true, y_pred):
          y_true = tf.dtypes.cast(y_true, tf.float32)
          y_pred = tf.dtypes.cast(y_pred, tf.float32)
#             y_pred=  tf.transpose(y_pred, perm=[1, 0,2])

          
          weight = tf.constant(class_weight, dtype=tf.float32)
          weight_per_sample = tf.transpose(tf.gather(weight, tf.argmax(y_true, axis=-1)))
          weight_per_sample = tf.expand_dims(weight_per_sample, axis=-1)
#             losses = tf.keras.losses.mean_squared_error(y_true, y_pred)
          losses = tf.math.square(y_true-y_pred)*weight_per_sample
          return tf.reduce_mean(losses, axis=-1)
  return loss

#### Pick out one channel for each sample
For now it takes the same channel for all samples

In [5]:
for layerOpt in layer_options:
    for extraOpt in extra_options:
        for weightOpt in weight_options:
            for sf in sfs:
                if weightOpt == 0:
                    using_weights = False
                else:
                    using_weights = True

                name = 'multi_channels_test' #name of model - should be descriptive
                save_folder = 'sf'+ str(sf) + '_layerOpt' + str(layerOpt) + extraOpt  #hyperparameter description her
                if using_weights == True:
                    save_folder = save_folder + '_W'
                    name = name + '_weighted'
                    if weightOpt == 2:
                        save_folder = save_folder + 'crazier'


                if weightOpt == 2:
                    class_weights = [1.0, 5.0, 36.0]
                else: 
                    class_weights = [1.0, 5.0, 10.0]

                class_weights = np.array([class_weights[i] for i in range(len(class_weights))])

                signal_size = 1

                y = layers.Input(shape=(channel_width,1), dtype='float32', name='Input')
                x = layers.Conv1D(sf, 3, padding='same', activation='relu', use_bias=True)(y)
                for iter in range(layerOpt-1):
                    x = layers.Conv1D(sf, 3, padding='same', activation='relu')(x)

                x = layers.MaxPool1D(pool_size=2,strides=2)(x)
                for iter in range(layerOpt):
                    x = layers.Conv1D(sf*2, 3, padding='same', activation='relu')(x)

                x = layers.MaxPool1D(pool_size=2,strides=2)(x)
                for iter in range(layerOpt):
                    x = layers.Conv1D(sf*4, 3, padding='same', activation='relu')(x)
                #x = layers.Conv1D(sf*4, 3, padding='same', activation='relu')(x)
                x = layers.MaxPool1D(pool_size=2,strides=2)(x)
                
                if extraOpt == 'b': 
                    for iter in range(layerOpt):
                        x = layers.Conv1D(sf*8, 3, padding='same', activation='relu')(x)
                    x = layers.MaxPool1D(pool_size=2,strides=2)(x)
                
                x = layers.Flatten()(x)
                


                class_layer = [{}]*channel_width
                output_layer = [{}]*channel_width
                for iter in range(channel_width):
                    class_layer[iter] = layers.Dropout(rate=0.3)(x)
                    if sf < 3:
                        class_layer[iter] = layers.Dense(3,activation='relu')(class_layer[iter])
                    else: 
                        class_layer[iter] = layers.Dense(sf,activation='relu')(class_layer[iter])
                    class_layer[iter] = layers.Dropout(rate=0.3)(class_layer[iter])
                    #class_layer[iter] = layers.Dense(64,activation='relu')(class_layer[iter])
                    output_layer[iter] = layers.Dense(nr_classes, activation='softmax', name=('out'+str(iter)))(class_layer[iter])



                model = Model(inputs=[y], outputs=[out_layer for out_layer in output_layer])


                isExist = os.path.exists(save_folder)
                if not isExist:
                    os.makedirs(save_folder)
                    print('Created "' + save_folder + '" directory')
                else:
                    print('"'+ save_folder + '" directory already existed - skipping')
                    continue


                nr_params = count_params(model.trainable_weights)

                cre = open(save_folder + '/' + 'Params_' + str(nr_params), 'x')


                # ------------- model compilation --------------
                ourAdam = Adam()
                optimizer = tf.keras.optimizers.RMSprop()

                if using_weights == True:
                        loss_func = weighted_mean_squared_error(class_weights)
                else:
                        loss_func = 'categorical_crossentropy'

                metric_T = 'accuracy'
                loss_dict = {}
                metric_dict = {}
                for iter in range(channel_width):
                        loss_dict['out'+str(iter)] = loss_func
                        metric_dict['out'+str(iter)] = metric_T



                model.compile(optimizer=ourAdam, loss = loss_dict,
                              metrics=metric_T) 


                if nr_params > 1000000:
                    ting = 1
                elif nr_params > 500000:
                    ting = 2
                elif nr_params > 250000:
                    ting = 4    
                elif nr_params > 125000:
                    ting = 8
                elif nr_params < 50000:
                    ting = 32
                elif nr_params < 10000:
                    ting = 64
                else:
                    ting = 16


                BATCH_SIZE = 160*ting
                EPOCH = 600 + 75 * ting
                
                if ting > 15:
                    ekstra = 15
                elif ting < 4:
                    ekstra = 0
                else:
                    ekstra = ting

                # Set the model training parameters
                # Stop model training when the training loss is not dropped
                callbacks_list = [callbacks.EarlyStopping(
                                        monitor='val_loss', 
                                        patience=math.floor(10 + ekstra), 
                                        verbose=0, 
                                        mode='auto',
                                        restore_best_weights=True,
                                    )
                                            ]

                # ------------- Starting model Training --------------

                hist = model.fit(data_train,[labels_train[:,iter,:] for iter in range(channel_width)],
                          batch_size = BATCH_SIZE, 
                          epochs = EPOCH, 
                          callbacks= callbacks_list,
                          verbose=0,
                          validation_split=0.25)


                # Show loss curves
                fig1 = plt.figure()
                plt.title('Training loss')
                plt.plot(hist.epoch, hist.history['loss'], label='train loss')
                plt.plot(hist.epoch, hist.history['val_loss'], label='val_loss')
                plt.legend()
                plt.savefig(save_folder + '/%s Training loss.pdf' %(name), format='pdf')
                #plt.show()
                plt.close(fig1)

                fig2 = plt.figure()
                plt.title('Training accuracy')
                plt.xlabel("Epoch #")
                plt.ylabel("Accuracy")

                lossNames = ['out0_accuracy', 'out'+str(math.floor(channel_width/3))+'_accuracy', 'out'+str(math.floor(channel_width*2/3))+'_accuracy']
                for (i, l) in enumerate(lossNames):
                    # plot the loss for both the training and validation data
                    title = "Loss for {}".format(l) if l != "loss" else "Total loss"
                    plt.plot(hist.epoch, hist.history[l], label=l)
                    plt.plot(hist.epoch, hist.history["val_" + l],
                        label="val_" + l)
                    plt.legend()
                plt.savefig(save_folder + '/%s Training acc.pdf' %(name), format='pdf')
                #plt.show()
                plt.close(fig2)

                evalDict = model.evaluate(data_test,[labels_test[:,iter,:] for iter in range(channel_width)])
                totalA = 0
                for i in range(channel_width+1,channel_width+1+channel_width):
                    totalA += evalDict[i]

                totalA /= channel_width
                print(totalA)

                # Saving dict of history and evaluation result
                with open(save_folder + '/' + 'histDict', 'wb') as file_pi:
                    pickle.dump(hist.history, file_pi)

                with open(save_folder + '/' + 'evalDict' + str(totalA), 'wb') as file_pi:
                    pickle.dump(evalDict, file_pi)

                model.save(save_folder + '/' + name + '_Model')

                #Test on test data
                true_test_labels = np.argmax(labels_test, axis=-1)
                test_predictions = model.predict(data_test)
                test_result = np.argmax(test_predictions, axis=-1).T

                #classification report
                class_names = ['Empty channel', 'Wi-Fi', 'Bluetooth']
                class_report = classification_report(true_test_labels.flatten(), test_result.flatten(),target_names=class_names)

                with open(save_folder + '/' + 'classReportString', 'wb') as file_pi:
                    pickle.dump(class_report, file_pi)

                #Confusion matric plot
                plt.figure()
                ConfusionMatrixDisplay.from_predictions(true_test_labels.flatten(), test_result.flatten(),normalize='true',cmap='Greens',colorbar=False,display_labels=class_names)
                plt.title('Confusion Matrix')
                plt.savefig(save_folder +'/confusion_matrix_'+ name +'.pdf', format='pdf')
                plt.close()

                class_reportDict = classification_report(true_test_labels.flatten(), test_result.flatten(),output_dict=True, target_names=class_names)
                for key in class_reportDict:
                    try:
                        class_reportDict[key]['Samples'] = class_reportDict[key].pop('support')
                    except Exception as e:
                        print(e)
                #print(class_reportDict)
                class_reportDict.pop('accuracy')
                df = pd.DataFrame(class_reportDict).transpose().round(decimals=3)
                dfi.export(df, save_folder + '/' +name + "_ClassReport.png", table_conversion="matplotlib")


2023-05-21 09:39:38.989979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-21 09:39:38.993358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-21 09:39:38.993468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

"sf1_layerOpt2a" directory already existed - skipping
"sf2_layerOpt2a" directory already existed - skipping
"sf4_layerOpt2a" directory already existed - skipping
"sf6_layerOpt2a" directory already existed - skipping
"sf8_layerOpt2a" directory already existed - skipping
"sf16_layerOpt2a" directory already existed - skipping
"sf32_layerOpt2a" directory already existed - skipping
"sf1_layerOpt2a_W" directory already existed - skipping
"sf2_layerOpt2a_W" directory already existed - skipping
"sf4_layerOpt2a_W" directory already existed - skipping
"sf6_layerOpt2a_W" directory already existed - skipping
"sf8_layerOpt2a_W" directory already existed - skipping
"sf16_layerOpt2a_W" directory already existed - skipping
"sf32_layerOpt2a_W" directory already existed - skipping
"sf1_layerOpt2a_Wcrazier" directory already existed - skipping
"sf2_layerOpt2a_Wcrazier" directory already existed - skipping
"sf4_layerOpt2a_Wcrazier" directory already existed - skipping
"sf6_layerOpt2a_Wcrazier" directory a

2023-05-21 09:40:52.683287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-21 09:40:53.601502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1817/1817 [==============================] - 42s 23ms/step - loss: 14.1715 - out0_loss: 0.0666 - out1_loss: 0.1703 - out2_loss: 0.1781 - out3_loss: 0.1586 - out4_loss: 0.1724 - out5_loss: 0.1383 - out6_loss: 0.2084 - out7_loss: 0.1745 - out8_loss: 0.1952 - out9_loss: 0.1535 - out10_loss: 0.2018 - out11_loss: 0.1696 - out12_loss: 0.2174 - out13_loss: 0.2022 - out14_loss: 0.1957 - out15_loss: 0.2059 - out16_loss: 0.2253 - out17_loss: 0.1825 - out18_loss: 0.2082 - out19_loss: 0.1914 - out20_loss: 0.2336 - out21_loss: 0.1828 - out22_loss: 0.2094 - out23_loss: 0.1932 - out24_loss: 0.2106 - out25_loss: 0.1920 - out26_loss: 0.2341 - out27_loss: 0.1834 - out28_loss: 0.2039 - out29_loss: 0.1875 - out30_loss: 0.2001 - out31_loss: 0.2325 - out32_loss: 0.2471 - out33_loss: 0.1977 - out34_loss: 0.2139 - out35_loss: 0.1971 - out36_loss: 0.2424 - out37_loss: 0.1837 - out38_loss: 0.2020 - out39_loss: 0.1898 - out40_loss: 0.2390 - out41_loss: 0.1979 - out42_loss: 0.2165 - out43_loss: 0.1901 - out44_los

2023-05-21 09:59:58.374883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-05-21 09:59:58.380371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-05-21 09:59:58.385360: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-05-21 09:59:58

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>